In [6]:
import tensorflow as tf
import os
import pandas as pd
import math
import numpy as np
import matplotlib.pylab as plt
import csv
PH = 5

def readData(filename):
    EntryData = []
    x_data = []
    y_data = []
    tmp_x = []
    tmp_y = []
    with open(filename,'r') as f:
        for line in f:
            EntryData.append(float(line))
    
    df = pd.Series(EntryData)

    while True:
        for i in df[0:7]:
            tmp_x.append([float(i)])
        tmp_y.append(float(df[7 + (PH/5)]))
        x_data.append(tmp_x)
        y_data.append(tmp_y)
        
        tmp_x = []
        tmp_y = []
        df = df.shift(-1)
        if(math.isnan(df[7+(PH/5)])):
            break
        
    if(len(x_data[-1]) != 7):
       xSize = 7-len(x_data[-1])
       for i in range(xSize):
           x_data[-1].append(0.0)
    if(len(y_data[-1])!=1):
       y_data[-1].append(0.0)
    
    x_data = np.asarray(x_data)
    y_data = np.asarray(y_data)
    data = [x_data,y_data]
    return data



filename = os.listdir("sch")

train_data_name = []
test_data_name = []

for fn in filename:
    if fn.find("test") != -1:
        test_data_name.append(fn)
    elif fn.find("train") != -1:
        train_data_name.append(fn)

#total_x_data = []
#total_y_data = []        

train_x_data = []
train_y_data = []

for fn in train_data_name:
    #if fn.find("885633") != -1:
        #print(fn+"제외")
    #elif fn.find("365303") != -1:
        #print(fn+"제외")
    #else:
    x,y = readData("sch/"+str(fn))
    train_x_data.append(x)
    #total_x_data.append(x)
    train_y_data.append(y)
    #total_y_data.append(y)


test_x_data = []
test_y_data = []
for fn in test_data_name:
    #if fn.find("885633") != -1:
        #print(fn+"제외")
    #elif fn.find("365303") != -1:
        #print(fn+"제외")
    #else:
    x,y = readData("sch/"+str(fn))
    test_x_data.append(x)
    #total_x_data.append(x)
    test_y_data.append(y)
    #total_y_data.append(y)
#print(A1cList)
#print(total_y_data)
#print(test_y_data[0])
print(test_data_name)

['test.csv']


In [2]:
X = tf.placeholder(tf.float32,shape=[None,7,1])
Y = tf.placeholder(tf.float32,shape=[None,1])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=15,state_is_tuple=True,activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)
Y_p = tf.contrib.layers.fully_connected(outputs[:,-1],1,activation_fn=None)

loss = tf.reduce_sum(tf.square(Y_p - Y))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train = optimizer.minimize(loss)

rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(Y, Y_p))))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for loop in range(len(train_x_data)):
    for i in  range(500):
        sess.run(train,feed_dict = {X:train_x_data[loop],Y:train_y_data[loop]})
        #sess.run(train,feed_dict = {X:total_x_data[loop],Y:total_y_data[loop]})
    print(str(loop / len(train_x_data) * 100) + "%")



0.0%


In [4]:
'''
pre_List = []
for loop in range(len(test_x_data)):
    for i,k in enumerate(test_x_data[loop]):
        ln = sess.run(Y_p,feed_dict={X:[k]})
        print("실제혈당 : "+str(test_y_data[0][i])+"예측혈당 : "+str(ln[0][0]))
        pre_List.append(ln[0][0])
        #print([sess.run(Y_p,feed_dict={X:[k]})[0][0],10.0])
    break;

'''
rmse_total = 0.0
rmse_one = 0.0
total_num = 0
one_person_num = 0
for size in range(len(test_x_data)):
    one_person_num = 0
    rmse_one = 0
    for i,k in enumerate(test_x_data[size]):
        total_num += 1
        one_person_num += 1
        #print(sess.run(rmse,feed_dict={X:[k],Y:[test_y_data[size][i]],A1c:[A1c_test[size][i]],G:[glucose_test[size][i]]}))
        tmp = sess.run(rmse,feed_dict={X:[k],Y:[test_y_data[size][i]]})
        rmse_total += tmp
        rmse_one += tmp
    print(str(size + 1)+"번째 환자 rmse : "+str(rmse_one/float(one_person_num)))

rmse_total = rmse_total / float(total_num)
print("전체 평균 rmse : "+str(rmse_total))

'''
plt.rcParams["figure.figsize"] = (20.4)
plt.plot(pre_List,color="red")
plt.plot(test_y_data[0],color="blue")
plt.show()
'''

[array([[[ 339.],
        [ 340.],
        [ 335.],
        [ 337.],
        [ 337.],
        [ 338.],
        [ 328.]],

       [[ 340.],
        [ 335.],
        [ 337.],
        [ 337.],
        [ 338.],
        [ 328.],
        [ 327.]],

       [[ 335.],
        [ 337.],
        [ 337.],
        [ 338.],
        [ 328.],
        [ 327.],
        [ 330.]],

       [[ 337.],
        [ 337.],
        [ 338.],
        [ 328.],
        [ 327.],
        [ 330.],
        [ 336.]],

       [[ 337.],
        [ 338.],
        [ 328.],
        [ 327.],
        [ 330.],
        [ 336.],
        [ 342.]],

       [[ 338.],
        [ 328.],
        [ 327.],
        [ 330.],
        [ 336.],
        [ 342.],
        [ 344.]],

       [[ 328.],
        [ 327.],
        [ 330.],
        [ 336.],
        [ 342.],
        [ 344.],
        [ 343.]],

       [[ 327.],
        [ 330.],
        [ 336.],
        [ 342.],
        [ 344.],
        [ 343.],
        [ 342.]],

       [[ 330.],
        [ 336.

'\nplt.rcParams["figure.figsize"] = (20.4)\nplt.plot(pre_List,color="red")\nplt.plot(test_y_data[0],color="blue")\nplt.show()\n'

In [ ]:
'''pre_List = []

for k in range(len(test_x_data)):
    pre = []
    for i,va in enumerate(test_x_data[0]):
    
        predict = sess.run(Y_p,feed_dict={X:[va],A:A1c_test[0]})
        #print(sess.run(Y_p,feed_dict={X:[va],A:A1c_test[9]}))
       # error = predict[0][0] - test_y_data[1][i]
        pre_List.append(predict[0][0])
    break;

plt.plot(pre_List,color = "red")
plt.plot(test_y_data[0],color = "blue")
plt.show()'''
        